<a href="https://colab.research.google.com/github/LakshmiMedapati96/DM-FinalProject/blob/main/DM_Project_Duplicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset

In [3]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! mkdir ~/.kaggle

In [5]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download tushar5harma/topredditcomments

 41% 9.00M/22.1M [00:00<00:00, 71.9MB/s]
100% 22.1M/22.1M [00:00<00:00, 103MB/s] 


In [7]:
! unzip topredditcomments.zip

Archive:  topredditcomments.zip
  inflating: Top_Posts.csv           
  inflating: Top_Posts_Comments.csv  


# The Coding begins...

# Importing Libraries, Loading and Merging the data

In [8]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

import string
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn import metrics

In [9]:
# Load the dataset
posts = pd.read_csv("Top_Posts.csv")
comments = pd.read_csv("Top_Posts_Comments.csv")

print("posts", posts.shape)
print("comments", comments.shape)

posts (2987, 10)
comments (223174, 2)


In [10]:
posts.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021


In [11]:
comments.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...


In [12]:
posts.subreddit.value_counts()

artificial         999
MachineLearning    998
datascience        990
Name: subreddit, dtype: int64

In [13]:
# The code in this cell is written by me by referring to Reference [1]

# Merging the 2 input files by using the common column "post_id" in both the files
merged_data = pd.merge(posts, comments, on="post_id", how="inner")

merged_data

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year,comment
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,The future 🤯
2,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Ohh the nightmare of making this into a stable...
...,...,...,...,...,...,...,...,...,...,...,...
223163,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,LiDAR is mot powerful sensor for the auto driv...
223164,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,So it can now idenrify traffic lights? Musk pr...
223165,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,Hydranet bro!
223166,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,It even shows flashing yellow turn arrows.


In [14]:
# Ignoring the unnecessary columns only
merged_data = merged_data[["comment", "subreddit"]]

merged_data.head()

,comment,subreddit
0,Twitter thread: [https://twitter.com/cyrildiag...,MachineLearning
1,The future 🤯,MachineLearning
2,Simple yet very useful. Thank you for sharing ...,MachineLearning
3,"Almost guaranteed, Apple will copy your idea i...",MachineLearning
4,Ohh the nightmare of making this into a stable...,MachineLearning


# Data Cleaning

In [15]:
print(merged_data.shape)

# checking for missing values
merged_data.isnull().sum()


(223168, 2)


comment      9
subreddit    0
dtype: int64

In [16]:
merged_data.dropna(inplace=True)

print(merged_data.shape)

(223159, 2)


<ipython-input-16-b1443c6d7364>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.dropna(inplace=True)


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
# Written by me by referring to Reference 
# Importing nltk library for in-built stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Written by me by referring to Reference 
# Importing nltk for Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Below lines of code are my own code
import string

# Defining a function to clean the data
def clean(doc):
    doc = str(doc)                                              # converting to string 
    doc = doc.lower().strip()                                   # converting to lower case and stripping the whitespaces

    doc = "".join([re.sub(r'^https?:\/\/.*[\r\n]*', '', doc, flags=re.MULTILINE) for char in doc]) 
    doc = "".join([re.sub('<.*?>+', '', doc) for char in doc])

    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])          # removing punctuations and digits

    
  
  # Written by me by referring to Reference 
    doc = " ".join([token for token in doc.split() if token not in stop_words])                              # splitting the doc into tokens and removing stopwords
  
  # Written by me by referring to Reference  
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])                                             # lemmatizing the doc
    return doc

In [ ]:
merged_data["comment"] = merged_data["comment"].apply(clean)

In [ ]:
merged_data.head()

,comment,subreddit
0,twitter thread httpstwittercomcyrildiagnestatu...,MachineLearning
1,future 🤯,MachineLearning
2,simple yet useful thank sharing code,MachineLearning
3,almost guaranteed apple copy idea,MachineLearning
4,ohh nightmare making stable product enough dri...,MachineLearning


In [ ]:
#data = merged_data.sample(frac=0.5)

In [ ]:
#data.shape

(111584, 2)

In [ ]:
# docs = list(merged_data['comment'])
# tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 3000) 
# tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)
# docs = tfidf_vectorizer_vectors.toarray()

In [ ]:
docs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Splitiing dataset

In [ ]:
X = docs 
y = merged_data['subreddit']

# Split the data into training and remaining datasets
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size = 0.6)

# Split the remaining dataset into development and test datasets
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, train_size = 0.5)


print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

ValueError: ignored

In [ ]:
# gnb = GaussianNB() 
# %time gnb.fit(X_train, y_train)

# y_pred_train = gnb.predict(X_train)
# y_pred_val = gnb.predict(X_val)
# y_pred_test = gnb.predict(X_test)

# print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
# print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))
# print("Validation Accuracy score:",accuracy_score(y_val, y_pred_val))

In [ ]:
# print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))

# Using MultiNormial Naive Bayes

In [ ]:
mnb = MultinomialNB() 
%time mnb.fit(X_train, y_train)

y_pred_train = mnb.predict(X_train)
y_pred_test = mnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


CPU times: user 1.43 s, sys: 38 ms, total: 1.47 s
Wall time: 1.03 s

Training Accuracy score: 0.6953979207743323
Testing Accuracy score: 0.6862789030292167


In [ ]:
print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.66      0.70      0.68      9531
    datascience       0.54      0.01      0.01      2019
     artificial       0.71      0.80      0.75     10766

       accuracy                           0.69     22316
      macro avg       0.63      0.50      0.48     22316
   weighted avg       0.67      0.69      0.65     22316



# Support Vector Classifier

In [ ]:
svc =  LinearSVC(class_weight='balanced') 
%time svc.fit(X_train, y_train)

y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Validation Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 5.09 s, sys: 27.2 ms, total: 5.12 s
Wall time: 5.36 s

Training Accuracy score: 0.6969550994801936
Testing Accuracy score: 0.6594819860189998


In [ ]:
print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.67      0.63      0.65      9531
    datascience       0.26      0.33      0.29      2019
     artificial       0.75      0.74      0.75     10766

       accuracy                           0.66     22316
      macro avg       0.56      0.57      0.56     22316
   weighted avg       0.67      0.66      0.66     22316



# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

y_pred_train = dtc.predict(X_train)
y_pred_val = dtc.predict(X_val)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Validation Accuracy score:",accuracy_score(y_val, y_pred_val))


Training Accuracy score: 0.9366075162812929
Testing Accuracy score: 0.6073669116329091


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=2)
%time rf.fit(X_train, y_train)

y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 27min 12s, sys: 2.54 s, total: 27min 15s
Wall time: 27min 14s

Training Accuracy score: 0.936715809284818
Testing Accuracy score: 0.6707295214196093


In [ ]:
print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))

                 precision    recall  f1-score   support

MachineLearning       0.63      0.70      0.67      9531
    datascience       0.36      0.05      0.08      2019
     artificial       0.71      0.76      0.74     10766

       accuracy                           0.67     22316
      macro avg       0.57      0.50      0.49     22316
   weighted avg       0.65      0.67      0.65     22316

